In [1]:
%load_ext autoreload
%autoreload 2
import numpy
import pandas
import io
import uuid
import psycopg.sql
import pyarrow
import pyarrow.parquet

import jobqueue
from jobqueue.connection_manager import ConnectionManager


In [2]:
# from enum import Enum, auto

# class ColumnSource(Enum):
#     run_column=auto()
#     run
#     run_column=auto()
#     run_column=auto()
#     run_column=auto()
#     run_column=auto()

from dataclasses import dataclass

    

@dataclass
class Column:
    name : str
    pandas_type : str
    

In [6]:

credentials = jobqueue.load_credentials('dmp')
with ConnectionManager(credentials) as connection:
    with connection.cursor(binary=True) as cursor:
        query = psycopg.sql.SQL("""
SELECT
    h.history history
FROM
    run_status s,
    run_data d,
    history h
WHERE TRUE
    AND s.id = d.id
    AND h.id = s.id
    AND s.id = 'ee9112e7-b896-4759-aec7-e4b2a43a9336'
LIMIT 1;
""")

        cursor.execute(query, binary=True)
        row = cursor.fetchone()
        with io.BytesIO(row[0]) as buffer:
            history = pyarrow.parquet.read_table(
                pyarrow.PythonFile(buffer, mode='r')).to_pandas()
        

history.describe()
history

,trained_loss,test_categorical_crossentropy,test_ms,validation_loss,trained_ms,train_loss,model_number,trained_categorical_crossentropy,retained,test_loss,...,model_epoch,epoch,epoch_startms_ms,train_accuracy,validation_categorical_crossentropy,masked_parameter_count,train_categorical_crossentropy,validation_accuracy,test_accuracy,free_parameter_count
0,0.074230,0.083588,247,0.065789,1366,0.187527,0,0.074230,True,0.083588,...,1,1,1690414433735,0.942513,0.065789,0,0.187527,0.977143,0.9734,61706
1,0.049490,0.063351,247,0.055764,1371,0.062463,0,0.049490,True,0.063351,...,2,2,1690414452583,0.981717,0.055764,0,0.062463,0.981714,0.9790,61706
2,0.041205,0.059407,248,0.060385,1379,0.042657,0,0.041205,True,0.059407,...,3,3,1690414464077,0.986991,0.060385,0,0.042657,0.979714,0.9812,61706
3,0.037530,0.057551,250,0.062414,1369,0.032554,0,0.037530,True,0.057551,...,4,4,1690414475456,0.989841,0.062414,0,0.032554,0.981429,0.9809,61706
4,0.022210,0.047054,250,0.046880,1380,0.025646,0,0.022210,True,0.047054,...,5,5,1690414486901,0.991717,0.046880,0,0.025646,0.986000,0.9848,61706
5,0.019490,0.048412,309,0.044977,1370,0.020964,0,0.019490,True,0.048412,...,6,6,1690414498411,0.993310,0.044977,0,0.020964,0.985429,0.9865,61706
6,0.022116,0.053896,250,0.054621,1375,0.017958,0,0.022116,True,0.053896,...,7,7,1690414509829,0.994071,0.054621,0,0.017958,0.984571,0.9839,61706
7,0.018714,0.056742,246,0.046640,1379,0.015007,0,0.018714,True,0.056742,...,8,8,1690414521188,0.995150,0.046640,0,0.015007,0.985714,0.9849,61706
8,0.023397,0.066820,245,0.062139,1384,0.013692,0,0.023397,True,0.066820,...,9,9,1690414532569,0.995239,0.062139,0,0.013692,0.984000,0.9830,61706
9,0.013412,0.057117,251,0.050795,1380,0.012175,0,0.013412,True,0.057117,...,10,10,1690414544013,0.995982,0.050795,0,0.012175,0.986286,0.9861,61706


In [7]:

credentials = jobqueue.load_credentials('dmp')
with ConnectionManager(credentials) as connection:
    with connection.cursor(binary=True) as cursor:
        query = psycopg.sql.SQL("""
SELECT
    by_epoch
FROM
    experiment2 e
WHERE TRUE
    AND e.experiment_id = 'b0d415cf-f228-c439-0480-3337c543c2e4'
LIMIT 1;
""")

        cursor.execute(query, binary=True)
        row = cursor.fetchone()
        with io.BytesIO(row[0]) as buffer:
            by_epoch = pyarrow.parquet.read_table(
                pyarrow.PythonFile(buffer, mode='r')).to_pandas()
        

by_epoch.describe()
by_epoch

,epoch,count,model_epoch_quantile_0,model_epoch_quantile_25,model_epoch_quantile_50,model_epoch_quantile_75,model_epoch_quantile_100,trained_categorical_crossentropy_quantile_0,trained_categorical_crossentropy_quantile_25,trained_categorical_crossentropy_quantile_50,...,test_accuracy_cmax_quantile_0,test_accuracy_cmax_quantile_25,test_accuracy_cmax_quantile_50,test_accuracy_cmax_quantile_75,test_accuracy_cmax_quantile_100,validation_accuracy_cmax_quantile_0,validation_accuracy_cmax_quantile_25,validation_accuracy_cmax_quantile_50,validation_accuracy_cmax_quantile_75,validation_accuracy_cmax_quantile_100
0,1,8,1.0,1.0,1.0,1.0,1.0,0.060437,0.061399,0.066523,...,0.9734,0.973925,0.97625,0.977575,0.9798,0.974000,0.975571,0.976571,0.977286,0.980571
1,2,8,2.0,2.0,2.0,2.0,2.0,0.039023,0.039946,0.041580,...,0.9762,0.980350,0.98225,0.982875,0.9843,0.980000,0.981071,0.981429,0.982143,0.985143
2,3,8,3.0,3.0,3.0,3.0,3.0,0.028624,0.030501,0.035552,...,0.9810,0.981500,0.98280,0.983900,0.9866,0.980857,0.981643,0.983000,0.984071,0.989429
3,4,8,4.0,4.0,4.0,4.0,4.0,0.019772,0.025258,0.027370,...,0.9812,0.981675,0.98450,0.985625,0.9885,0.980857,0.983214,0.983857,0.986857,0.989429
4,5,8,5.0,5.0,5.0,5.0,5.0,0.017522,0.022112,0.024034,...,0.9816,0.984450,0.98495,0.985650,0.9885,0.980857,0.983929,0.985429,0.987071,0.989429
5,6,8,6.0,6.0,6.0,6.0,6.0,0.016902,0.019114,0.019577,...,0.9819,0.984825,0.98520,0.986550,0.9885,0.984000,0.984786,0.985571,0.987071,0.989429
6,7,8,7.0,7.0,7.0,7.0,7.0,0.014243,0.016146,0.017878,...,0.9851,0.985725,0.98655,0.988050,0.9885,0.984000,0.985143,0.986000,0.987214,0.989429
7,8,8,8.0,8.0,8.0,8.0,8.0,0.010488,0.012256,0.017983,...,0.9851,0.986250,0.98745,0.988500,0.9888,0.984000,0.985143,0.986143,0.987214,0.989714
8,9,8,9.0,9.0,9.0,9.0,9.0,0.010544,0.011973,0.016272,...,0.9851,0.986250,0.98775,0.988500,0.9891,0.984000,0.985143,0.986143,0.987214,0.990000
9,10,8,10.0,10.0,10.0,10.0,10.0,0.008422,0.011188,0.012371,...,0.9859,0.986800,0.98775,0.988500,0.9897,0.984000,0.985143,0.986714,0.988643,0.990000


In [38]:
df.columns

Index(['experiment_id', 'old_experiment_id', 'experiment_attrs', 'run_id',
       'job_id', 'slurm_job_id', 'run_timestamp', 'seed', 'task_version',
       'num_nodes',
       ...
       'precision', 'worker_id', 'record_model', 'record_times',
       'python_version', 'record_metrics', 'tensorflow_version',
       'tensorflow_strategy', 'record_post_training_metrics',
       'make_batch_optimizer_butter_growth_eagle_gpu_1'],
      dtype='object', length=105)

In [40]:
df['run_history'].iloc[0]['train_loss']


0      1.247540
1      0.898057
2      0.847315
3      0.850349
4      0.847144
         ...   
145    0.225552
146    0.213775
147    0.213580
148    0.213284
149    0.210847
Name: train_loss, Length: 150, dtype: float32

In [42]:
run_df = df['run_history'].iloc[0]
run_df['trained_loss'] - run_df['train_loss']

0      0.000000
1     -0.047707
2      0.003034
3      0.000000
4     -0.001494
         ...   
145    0.026551
146   -0.015600
147   -0.015426
148   -0.008199
149    0.011152
Length: 150, dtype: float32